In [1]:
### Utenti Validi
# '2c938084683d9f87016d1636f55a00de': (u'Grazia Benedetti',True,"24.10.19","1.4.20")
# '2c938084683d9f87016ff0d54c3d0138': (u'Maria Luisa  Bortoluzzi',True,"29.1.20","1.5.20")
# '2c938084683d9f87016fd2ac0d240134': (u'Silvia Francescangeli',True,"23.1.20","1.5.20")
# '2c938084683d9f87016cd1cbb80100a7': (u'Enrico Vottero',True,"30.09.19","20.12.19")
# '2c938084683d9f87016d163666b900db': (u'Giorgio Bertolani',True,"9.10.19","20.12.19")}

### Setup delle librerie e connessione al database

In [2]:
import pymongo 
from pymongo import MongoClient
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import math

import pickle

In [31]:
myclient = pymongo.MongoClient("mongodb://localhost:27018/")
mydb = myclient["mls"]
mycol = mydb["mqttlogs"]

utente='2c938084683d9f87016d163666b900db'
print("PROCESSING USER:\n" + utente)
query = {'userid':utente}
db_entries = mycol.find(query)

PROCESSING USER:
2c938084683d9f87016d163666b900db


### Recupero di interventions e acks

In [32]:
ordered_interventions_id = []
interventions = dict()
acks = dict()

escape_interventions = ["641056j1-0g78-vjqf-cpju-7dnx895011y9"]

In [33]:
print("start loading for interventions and acks from file")
data_file = open("./data_interventions_"+str(utente)+".pkl", "rb")
ordered_interventions_id,interventions,acks,dangling_interventions,times = pickle.load(data_file)
data_file.close()

start loading for interventions and acks from file


### Selezione delle interventions contenenti l'elenco ordinato di stanze dove cercare l'utente

In [34]:
num = 0
intv_rooms = dict()

In [35]:
for i in interventions : 
    if  'rooms' in interventions[i]['raw_payload']['data'] : # i not in dangling_interventions and
        intv_rooms[i] = interventions[i]
        intv_rooms[i]["intervention_number"] = num
    num += 1

In [36]:
len(intv_rooms), intv_rooms

(21,
 {'07f15765-be75-4637-9b97-f263c3a5697d': {'_id': ObjectId('5d9dd7c7ebaf7e0a71dac5cd'),
   'topic': 'movecare/interventions/2c938084683d9f87016d163666b900db/INFO',
   'category': 'interventions',
   'userid': '2c938084683d9f87016d163666b900db',
   'raw_payload': {'userid': '2c938084683d9f87016d163666b900db',
    'id': '07f15765-be75-4637-9b97-f263c3a5697d',
    'time': {'temporality': 'timestamp', 't': 1570625479.3155713},
    'data': {'objectid': '000000000000000000000AA3',
     'rooms': [],
     'code': 'OS'},
    'ivcode': 'INFO',
    'docking': False,
    'priority': 5},
   'created_at': datetime.datetime(2019, 10, 9, 12, 51, 19, 321000),
   'updatedAt': datetime.datetime(2019, 10, 9, 12, 51, 19, 321000),
   '__v': 0,
   'intervention_number': 1},
  '5a4b9989-699e-4a70-9165-304744f44d06': {'_id': ObjectId('5d9dd863ebaf7e0a71dac65f'),
   'topic': 'movecare/interventions/2c938084683d9f87016d163666b900db/INFO',
   'category': 'interventions',
   'userid': '2c938084683d9f87016d163

### Salvataggio del risultato

In [37]:
data_file = open("./roomlist_interventions_"+str(utente)+".pkl", "wb")
pickle.dump(intv_rooms,data_file)
data_file.close()